### WEATHER API

In [ ]:
import requests
import numpy as np
import pandas as pd
API_KEY = ""
CITY = "Boston,US"
WEATHER_URL = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={CITY}&aqi=yes"

In [ ]:
response = requests.get(WEATHER_URL)
response.status_code

In [ ]:
data = response.json()["current"]
time = data["last_updated"]
temperature = data["temp_c"]
precip_mm = data["precip_mm"]
humidity = data["humidity"]
wind_kph = data["wind_kph"]
aq = data["air_quality"]

In [ ]:
print(time)
print(temperature)
print(precip_mm)
print(humidity)
print(wind_kph)
print(aq)

In [ ]:
ALERTS_URL = f"http://api.weatherapi.com/v1/alerts.json?key={API_KEY}&q={CITY}"
response = requests.get(ALERTS_URL)
response.status_code
alerts = response.json()["alerts"]["alert"]


### MBTA API

In [ ]:
import requests
import numpy as np
import pandas as pd

MBTA_URL = "https://api-v3.mbta.com/"
VEHICLES = "vehicles?filter[route_type]=3"
SCHEDULE = lambda trip, stop: f"schedules?filter[trip]={trip}&filter[stop]={stop}"
PREDICTION = lambda trip, stop: f"predictions?filter[trip]={trip}&filter[stop]={stop}"
ROUTES = "routes?filter[type]=3"

In [ ]:
response = requests.get(MBTA_URL + VEHICLES)
print(f"status: {response.status_code}")

if response.status_code == 200:
    vehicles = response.json()["data"]
    print(vehicles[0])
    print(f"number of vehicles {len(vehicles)}")

    vehicles_per_route = {}
    for v in vehicles:
        vid = v["id"]
        route = v["relationships"]["route"]["data"]["id"]

        if route not in vehicles_per_route.keys():
            vehicles_per_route[route] = [vid]
        else:
            vehicles_per_route[route].append(vid)

    print(f"number of routes {len(vehicles_per_route.keys())}")

    data = []
    for r, vehicles in vehicles_per_route.items():
        for v in vehicles:
            data.append((r, v))

    df = pd.DataFrame(data, columns=["route_id", "vehicle_id"])
    df.to_csv("routes_vehicles.csv", index=False)

In [ ]:
response = requests.get(MBTA_URL + ROUTES)

In [ ]:
routes = response.json()["data"]
print(f"number of routes {len(routes)}")

routes = set([r["id"] for r in routes])

In [ ]:
df = pd.read_csv("routes_vehicles.csv")
print(routes.difference(set(df["route_id"].values)))
print(set(df["route_id"].values).difference(routes))

In [ ]:
sorted_grs = df.groupby("route_id").size().sort_values(ascending=False)
top = list(sorted_grs.head(11).items())[1:]
routes_to_plot = [x[0] for x in top] 

In [ ]:
from datetime import datetime
import time

response = requests.get(MBTA_URL + VEHICLES)
print(f"status: {response.status_code}")

def get_time_diff(t1, t2):
    t1 = datetime.fromisoformat(t1)
    t2 = datetime.fromisoformat(t2)    
    diff = t1 - t2

    return diff.total_seconds() / 60

if response.status_code == 200:
    vehicles = response.json()["data"]

    # chose first vehicle
    vehicle = vehicles[1]
    vid = vehicle["id"]
    
    print(vid)
    trip_id = vehicle["relationships"]["trip"]["data"]["id"]
    stop_id = vehicle["relationships"]["stop"]["data"]["id"]

    headers = {
        "Accept": "application/json",
    }
    
    sch = requests.get(MBTA_URL + SCHEDULE(trip_id, stop_id), headers=headers)
    print(response.headers)
   
    pred = requests.get(MBTA_URL + PREDICTION(trip_id, stop_id), headers=headers)
    
    sch = sch.json()["data"][0]["attributes"]
    pred = pred.json()["data"][0]["attributes"]

    target = "arrival_time"
    if sch[target] == None:
        target = "departure_time"

    diff = get_time_diff(pred[target], sch[target])
    
    if diff > 0:
        print(f"delay is {diff} min")
    else:
        print(f"is eraly by {diff} min")

    if response.status_code == 200:
        data = response.json()["data"]
        print(len(data))    
        print(data)

In [ ]:
import folium

sorted_grs = df.groupby("route_id").size().sort_values(ascending=False)
top = list(sorted_grs.head(6).items())[1:]
routes = [x[0] for x in top] 
colors = ["red", "blue", "green", "orange", "purple"]

def get_stops(rid):
    response = requests.get(MBTA_URL + f"stops?filter[route]={rid}")
    sts = response.json()["data"]
    stops = []
    for stop in sts:
        stops.append((stop["attributes"]["latitude"], stop["attributes"]["longitude"]))
    return stops

def plot_routes(routes, colors):
    stop = get_stops(routes[0])[0]
    route_map = folium.Map(location=stop, zoom_start=13)
    for rid, color in zip(routes, colors):    
        folium.PolyLine(locations=get_stops(rid), color=color, weight=5, opacity=0.8).add_to(route_map)
        route_map.save("routes.html")

plot_routes(routes, colors)
